importing libraries

In [1]:
import pymysql  # Make sure to import the driver
from sqlalchemy import create_engine, text
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

configuring the connection to the database

In [2]:
# Database credentials
username = "root"
password = "abdul256"
host = "localhost"
port = "3306"
database = "sqlbasic"

# Create the engine with pymysql
engine = create_engine(f"mysql+pymysql://{username}:{password}@{host}:{port}/{database}")

# helper functin select
def update(query):
    try:
        return pd.read_sql(query,  con = engine)
    except:
        print("succefull update")
        
def select(query):
        return pd.read_sql(query,  con = engine)


In [7]:
df = pd.read_csv("datasets/financial_loan.csv")

In [10]:

df.to_sql("loanapplications", engine, index=False, if_exists="replace")

38576

In [3]:
# lets look at our data
query = '''SELECT * FROM loanapplications
           LIMIT 5'''
select(query)

,id,address_state,application_type,emp_length,emp_title,grade,home_ownership,issue_date,last_credit_pull_date,last_payment_date,loan_status,next_payment_date,member_id,purpose,sub_grade,term,verification_status,annual_income,dti,installment,int_rate,loan_amount,total_acc,total_payment,Month
0,1077430,GA,INDIVIDUAL,< 1 year,others,C,RENT,2021-02-11,13-09-2021,13-04-2021,Charged Off,13-05-2021,1314167,car,C4,60 months,Source Verified,30000.0,0.0100,59.83,0.1527,2500,4,1009,February
1,1072053,CA,INDIVIDUAL,9 years,others,E,RENT,2021-01-01,14-12-2021,15-01-2021,Fully Paid,15-02-2021,1288686,car,E1,36 months,Source Verified,48000.0,0.0535,109.43,0.1864,3000,4,3939,January
2,1069243,CA,INDIVIDUAL,4 years,Technologies,C,RENT,2021-01-05,12-12-2021,09-01-2021,Charged Off,09-02-2021,1304116,car,C5,36 months,Not Verified,50000.0,0.2088,421.65,0.1596,12000,11,3522,January
3,1041756,TX,INDIVIDUAL,< 1 year,others,B,MORTGAGE,2021-02-25,12-12-2021,12-03-2021,Fully Paid,12-04-2021,1272024,car,B2,60 months,Source Verified,42000.0,0.0540,97.06,0.1065,4500,9,4911,February
4,1068350,IL,INDIVIDUAL,10+ years,Corporatio,A,MORTGAGE,2021-01-01,14-12-2021,15-01-2021,Fully Paid,15-02-2021,1302971,car,A1,36 months,Verified,83000.0,0.0231,106.53,0.0603,3500,28,3835,January


- Total Loan Applications: We need to calculate the total number of loan applications \
received during a specified period. Additionally, it is essential to monitor the Month-to-Date (MTD) \
Loan Applications and track changes Month-over-Month (MoM).


First of all let's make sure that columns issue_date has a proper DATA type

In [140]:
query = '''UPDATE loanapplications
          SET issue_date = STR_TO_DATE(issue_date, '%%Y-%%m-%%d');'''
update(query)

succefull update


In [163]:
query = '''SELECT COUNT(id) as total_applications FROM loanapplications 
           WHERE MONTH(issue_date) = 12 AND YEAR(issue_date) = 2021;'''
select(query)


,total_applications
0,4314


- Total Funded Amount by specifc date

In [170]:
query = '''SELECT SUM(loan_amount) as total_amount_funded FROM loanapplications
           WHERE MONTH(issue_date) = 12 AND YEAR(issue_date) = 2021;'''
select(query)

,total_amount_funded
0,53981425.0


- Total Amount Received by specific date

In [171]:
query = '''SELECT SUM(total_payment) as total_amount_recieved FROM loanapplications
           WHERE MONTH(issue_date) = 12 AND YEAR(issue_date) = 2021; '''
select(query)

,total_amount_recieved
0,58074380.0


- Average Interest Rate 

In [174]:
query = '''SELECT ROUND(AVG(int_rate), 2) * 100 as avg_APR FROM loanapplications
           WHERE MONTH(issue_date) = 12 AND YEAR(issue_date) = 2021; '''
select(query)

,avg_APR
0,12.0


- Average Debt-to-Income Ratio (DTI)

In [180]:
query = '''SELECT ROUND(AVG(dti),2) * 100 as avg_dti FROM loanapplications
           WHERE MONTH(issue_date) = 12 AND YEAR(issue_date) = 2021; '''
select(query)

,avg_dti
0,14.0


- Good Loan Application - includes loans with a loan status of 'Fully Paid' and 'Current.'.
- Bad Loan Application -  includes loans with a loan status of 'Charged Off.'

- Good Loan Total Received Amount:

In [191]:
query = '''SELECT SUM(total_payment) AS total_loan_amount FROM loanapplications
           WHERE loan_status = 'Fully Paid' OR loan_status = 'Current'  '''
select(query)

,total_loan_amount
0,435786170.0


In [207]:
query = '''SELECT loan_status, 
           COUNT(loan_status) as TotalNumber,
           SUM(loan_amount) AS total_funded,
           SUM(total_payment) AS total_recieved,
           ROUND(AVG(int_rate), 2) * 100 AS average_APR,
           ROUND(AVG(dti), 2) * 100 AS average_dti
           FROM loanapplications
           GROUP BY loan_status
           '''
select(query)

,loan_status,TotalNumber,total_funded,total_recieved,average_APR,average_dti
0,Charged Off,5333,65532225.0,37284763.0,14.0,14.0
1,Fully Paid,32145,351358350.0,411586256.0,12.0,13.0
2,Current,1098,18866500.0,24199914.0,15.0,15.0


In [206]:
query = '''SELECT loan_status, 
           SUM(loan_amount) AS total_funded,
           SUM(total_payment) AS total_recieved
           FROM loanapplications
           WHERE MONTH(issue_date) = 12
           GROUP BY loan_status
           '''
select(query)

,loan_status,total_funded,total_recieved
0,Fully Paid,41302025.0,47815851.0
1,Charged Off,8732775.0,5324211.0
2,Current,3946625.0,4934318.0


Tableau queries for visualization

- 1. Monthly Trends by Issue Date (Line Chart):

In [222]:
query = '''SELECT DATE_FORMAT(issue_date, '%%M') as month,
                  COUNT(id) as total_applications,
                  SUM(loan_amount) AS total_funded,
                  SUM(total_payment) AS total_recieved
            FROM loanapplications
            GROUP BY MONTH(issue_date), DATE_FORMAT(issue_date, '%%M')
            ORDER BY MONTH(issue_date)'''
select(query)

,month,total_applications,total_funded,total_recieved
0,January,2332,25031650.0,27578836.0
1,February,2279,24647825.0,27717745.0
2,March,2627,28875700.0,32264400.0
3,April,2755,29800800.0,32495533.0
4,May,2911,31738350.0,33750523.0
5,June,3184,34161475.0,36164533.0
6,July,3366,35813900.0,38827220.0
7,August,3441,38149600.0,42682218.0
8,September,3536,40907725.0,43983948.0
9,October,3796,44893800.0,49399567.0


- Regional Analysis by State (Filled Map):

In [236]:
query = '''SELECT address_state,
                  COUNT(id) as total_applications,
                  SUM(loan_amount) AS total_funded,
                  SUM(total_payment) AS total_recieved
            FROM loanapplications
            GROUP BY address_state
            ORDER BY SUM(loan_amount) DESC
            LIMIT 5
            '''
select(query)

,address_state,total_applications,total_funded,total_recieved
0,CA,6894,78484125.0,83901234.0
1,NY,3701,42077050.0,46108181.0
2,TX,2664,31236650.0,34392715.0
3,FL,2773,30046125.0,31601905.0
4,NJ,1822,21657475.0,23425159.0


- Loan Term Analysis (Donut Chart):

In [239]:
query = '''SELECT term,
                  COUNT(id) as total_applications,
                  SUM(loan_amount) AS total_funded,
                  SUM(total_payment) AS total_recieved
            FROM loanapplications
            GROUP BY term
            ORDER BY SUM(loan_amount) DESC
            '''
select(query)

,term,total_applications,total_funded,total_recieved
0,36 months,28237,273041225.0,294709458.0
1,60 months,10339,162715850.0,178361475.0


 Employee Length Analysis (Bar Chart):

In [241]:
query = '''SELECT * FROM loanapplications
           LIMIT 5'''
select(query)

,id,address_state,application_type,emp_length,emp_title,grade,home_ownership,issue_date,last_credit_pull_date,last_payment_date,loan_status,next_payment_date,member_id,purpose,sub_grade,term,verification_status,annual_income,dti,installment,int_rate,loan_amount,total_acc,total_payment
0,1077430,GA,INDIVIDUAL,< 1 year,Ryder,C,RENT,2021-02-11,13-09-2021,13-04-2021,Charged Off,13-05-2021,1314167,car,C4,60 months,Source Verified,30000.0,0.0100,59.83,0.1527,2500,4,1009
1,1072053,CA,INDIVIDUAL,9 years,MKC Accounting,E,RENT,2021-01-01,14-12-2021,15-01-2021,Fully Paid,15-02-2021,1288686,car,E1,36 months,Source Verified,48000.0,0.0535,109.43,0.1864,3000,4,3939
2,1069243,CA,INDIVIDUAL,4 years,Chemat Technology Inc,C,RENT,2021-01-05,12-12-2021,09-01-2021,Charged Off,09-02-2021,1304116,car,C5,36 months,Not Verified,50000.0,0.2088,421.65,0.1596,12000,11,3522
3,1041756,TX,INDIVIDUAL,< 1 year,barnes distribution,B,MORTGAGE,2021-02-25,12-12-2021,12-03-2021,Fully Paid,12-04-2021,1272024,car,B2,60 months,Source Verified,42000.0,0.0540,97.06,0.1065,4500,9,4911
4,1068350,IL,INDIVIDUAL,10+ years,J&J Steel Inc,A,MORTGAGE,2021-01-01,14-12-2021,15-01-2021,Fully Paid,15-02-2021,1302971,car,A1,36 months,Verified,83000.0,0.0231,106.53,0.0603,3500,28,3835


In [249]:
query = ''' SELECT loan_status, COUNT(loan_status) / SUM()  as count FROM loanapplications
            GROUP BY loan_status'''
select(query)

,loan_status,count
0,Charged Off,None
1,Fully Paid,None
2,Current,None


In [272]:
query = '''SELECT loan_status, ROUND(COUNT(*) / (SELECT COUNT(*) FROM loanapplications), 3) * 100 AS percentage FROM loanapplications
           GROUP BY loan_status'''
select(query)

,loan_status,percentage
0,Charged Off,13.8
1,Fully Paid,83.3
2,Current,2.8


- Add new feature - Month

In [12]:
query = '''ALTER TABLE loanapplications
           ADD COLUMN Month VARCHAR(255);'''
update(query)

succefull update


In [15]:
query = '''UPDATE loanapplications
           SET Month = DATE_FORMAT(issue_date, '%%M') '''
update(query)

succefull update


In [16]:
query = '''SELECT Month FROM loanapplications'''
select(query)

,Month
0,February
1,January
2,January
3,February
4,January
...,...
38571,July
38572,October
38573,September
38574,October


In [4]:
query = '''SELECT emp_title, COUNT(emp_title) as count FROM loanapplications 
           GROUP BY emp_title
           LIMIT 1,5
           '''
select(query)

,emp_title,count
0,Technologies,545
1,Corporatio,4178
2,Self-employed,194
3,School,1093
4,Gov,410


- EmploymentTitle column has too much unique values lets try to categorize them

In [145]:
query =  '''UPDATE loanapplications
            SET emp_title = CASE
             WHEN emp_title LIKE '%%care%%' THEN 'HealthCare'
             WHEN emp_title LIKE '%%engine%%' THEN 'Engineering'
             WHEN emp_title LIKE '%%Tech%%' THEN 'Technologies'
             WHEN emp_title LIKE '%%Corp%%' THEN 'Corporatio'
             WHEN emp_title LIKE '%%self%%' THEN 'Self-employed'
             WHEN emp_title LIKE '%%Market%%' THEN 'Marketing'
             WHEN emp_title LIKE '%%Police%%' THEN 'Police'
             WHEN emp_title LIKE '%%Bank%%' THEN 'Banking'
             WHEN emp_title LIKE '%%school%%' THEN 'School'
             WHEN emp_title LIKE '%%med%%' THEN 'HospitalEmployee'
             WHEN emp_title LIKE '%%Univer%%' THEN 'Universtity'
             WHEN emp_title LIKE '%%None%%' THEN 'None'
             WHEN emp_title LIKE '%%USA%%' THEN 'Gov'
             WHEN emp_title LIKE '%%gov%%' THEN 'Gov'
             WHEN emp_title LIKE '%%inc%%' THEN 'Corporation'
            ELSE 'others'
            END;
             '''  
update(query)

succefull update


In [163]:
query = '''WITH DepartmentRank AS (
        SELECT purpose, COUNT(*) as purpose_count
        FROM loanapplications
        GROUP BY purpose
        ORDER BY purpose_count DESC
        LIMIT 5)
        
        SELECT DISTINCT purpose, 
          CASE
            WHEN purpose IN (SELECT purpose FROM DepartmentRank) THEN 'TRUE'
            ELSE 'FALSE'
         END AS TopFiveFlag
        FROM loanapplications
        '''

select(query)

,purpose,TopFiveFlag
0,car,FALSE
1,credit card,TRUE
2,Debt consolidation,TRUE
3,educational,FALSE
4,home improvement,TRUE
5,house,FALSE
6,major purchase,TRUE
7,medical,FALSE
8,moving,FALSE
9,other,TRUE


In [179]:
query = '''SELECT purpose, COUNT(*) as purpose_count
        FROM loanapplications
        GROUP BY purpose
        ORDER BY purpose_count DESC
        LIMIT 6
           '''
select(query)

,purpose,purpose_count
0,Debt consolidation,18214
1,credit card,4998
2,other,3824
3,home improvement,2876
4,major purchase,2110
5,small business,1776


In [ ]:
query = '''SELECT Month, COUNT() FROM loanapplications '''

In [10]:
query = '''SELECT DISTINCT grade FROM loanapplications '''
select(query)

,grade
0,C
1,E
2,B
3,A
4,D
5,F
6,G


In [20]:
query = '''SELECT grade, 
           SUM(CASE WHEN loan_status = 'Current' THEN 1 ELSE 0 END) AS Current,
           SUM(CASE WHEN loan_status = 'Charged Off' THEN 1 ELSE 0 END) AS Charged_Off,
           SUM(CASE WHEN loan_status = 'Fully Paid' THEN 1 ELSE 0 END) AS Approved
           FROM loanapplications
           GROUP BY grade'''
select(query)

,grade,Current,Charged_Off,Approved
0,C,257.0,1266.0,6381.0
1,E,175.0,691.0,1920.0
2,B,327.0,1343.0,10004.0
3,A,35.0,552.0,9102.0
4,D,216.0,1072.0,3894.0
5,F,71.0,311.0,646.0
6,G,17.0,98.0,198.0


In [8]:
df.head(5)

,id,address_state,application_type,emp_length,emp_title,grade,home_ownership,issue_date,last_credit_pull_date,last_payment_date,loan_status,next_payment_date,member_id,purpose,sub_grade,term,verification_status,annual_income,dti,installment,int_rate,loan_amount,total_acc,total_payment
0,1077430,GA,INDIVIDUAL,< 1 year,Ryder,C,RENT,11-02-2021,13-09-2021,13-04-2021,Charged Off,13-05-2021,1314167,car,C4,60 months,Source Verified,30000.0,0.0100,59.83,0.1527,2500,4,1009
1,1072053,CA,INDIVIDUAL,9 years,MKC Accounting,E,RENT,01-01-2021,14-12-2021,15-01-2021,Fully Paid,15-02-2021,1288686,car,E1,36 months,Source Verified,48000.0,0.0535,109.43,0.1864,3000,4,3939
2,1069243,CA,INDIVIDUAL,4 years,Chemat Technology Inc,C,RENT,05-01-2021,12-12-2021,09-01-2021,Charged Off,09-02-2021,1304116,car,C5,36 months,Not Verified,50000.0,0.2088,421.65,0.1596,12000,11,3522
3,1041756,TX,INDIVIDUAL,< 1 year,barnes distribution,B,MORTGAGE,25-02-2021,12-12-2021,12-03-2021,Fully Paid,12-04-2021,1272024,car,B2,60 months,Source Verified,42000.0,0.0540,97.06,0.1065,4500,9,4911
4,1068350,IL,INDIVIDUAL,10+ years,J&J Steel Inc,A,MORTGAGE,01-01-2021,14-12-2021,15-01-2021,Fully Paid,15-02-2021,1302971,car,A1,36 months,Verified,83000.0,0.0231,106.53,0.0603,3500,28,3835


In [4]:
query = '''SELECT home_ownership, ROUND(AVG(annual_income), 2) as avg_income FROM loanapplications
           GROUP BY home_ownership'''
select(query)

,home_ownership,avg_income
0,RENT,57878.41
1,MORTGAGE,83763.04
2,OWN,60465.27
3,OTHER,71309.71
4,NONE,80733.33
